# File structure

This project assumes the following initial file structure:

```
.\main                       
|   applicationsPlots.ipynb  
|   applicationsData.json    
|   environment.yml          
|                            
\---images                   
    ...                      
```

# Modules

## Imports

In [ ]:
import json
import os
import types
from collections import defaultdict

import numpy as np
import pandas as pd
import pkg_resources
import plotly.graph_objects as go
from matplotlib.colors import CSS4_COLORS

## Versions

The current module versions in use are as follows:

In [ ]:
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        poorly_named_packages = {"PIL": "pillow", "sklearn": "scikit-learn"}
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name


imports = list(set(get_imports()))

requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name != "pip":
        requirements.append((m.project_name, m.version))

pd.DataFrame(requirements, columns=["Module", "Version"])

# Global variables

The following global variables are used throughout the entirety of the methodology section.

In [ ]:
APLICATION_DATA_PATH = os.path.join("data", "applications.json")

# Main

In [ ]:
def css_color_to_rgba(color_name, alpha):
    try:
        hex_value = CSS4_COLORS[color_name.lower()]
        rgb = tuple(int(hex_value[i : i + 2], 16) for i in (1, 3, 5))
        rgba = (*rgb, alpha)
        return f"rgba{rgba}"
    except KeyError:
        return None

In [ ]:
def create_neighbor_pairs(input_list):
    return [(input_list[i], input_list[i + 1]) for i in range(len(input_list) - 1)]

In [ ]:
with open(APLICATION_DATA_PATH, "r") as file:
    applications = json.load(file)

In [ ]:
labels = set()
links = defaultdict(int)

for application in applications:
    application["links"] = [
        (application["process"][index], application["process"][index + 1])
        for index in range(len(application["process"]) - 1)
    ]
    labels.update(application["process"])

    for link in application["links"]:
        links[link] += 1

labels = list(labels)
edges = np.array(list(links.keys()))

source = [labels.index(sourceLabel) for sourceLabel in edges[:, 0]]
target = [labels.index(targetLabel) for targetLabel in edges[:, 1]]

nodeColors = [
    (
        "lightgreen"
        if node not in ["Rejected", "No Answer", "Declined"]
        else "red" if node in ["Rejected", "No Answer"] else "yellow"
    )
    for node in labels
]

linkColors = [
    (
        css_color_to_rgba("lightgreen", 0.5)
        if node not in ["Rejected", "No Answer", "Declined"]
        else (
            css_color_to_rgba("red", 0.5)
            if node in ["Rejected", "No Answer"]
            else css_color_to_rgba("yellow", 0.5)
        )
    )
    for node in edges[:, 1]
]

value = list(links.values())

In [ ]:
link = dict(source=source, target=target, value=value, color=linkColors)
node = dict(label=labels, pad=35, thickness=10, color=nodeColors)
data = go.Sankey(link=link, node=node)
fig = go.Figure(data=[data])

fig.show()